In [2]:
import pandas as pd
import numpy as np

def fusionner_csv_galaxies(fichier_cena, fichier_edd, fichier_sortie):
    """
    Fusionne deux fichiers CSV de données de galaxies.
    
    Parameters:
    - fichier_cena: chemin vers CenA_dwarfs.csv
    - fichier_edd: chemin vers EDD_data_TRGB.csv  
    - fichier_sortie: chemin pour le fichier CSV fusionné
    
    Returns:
    - DataFrame fusionné
    """
    
    # Lecture des fichiers CSV
    print("Lecture des fichiers CSV...")
    df_cena = pd.read_csv(fichier_cena)
    df_edd = pd.read_csv(fichier_edd)
    
    print(f"CenA_dwarfs.csv: {len(df_cena)} lignes")
    print(f"EDD_data_TRGB.csv: {len(df_edd)} lignes")
    
    # Harmonisation des noms de colonnes pour EDD
    df_edd = df_edd.rename(columns={
        'Ra (en degrees)': 'RA',
        'Dec (en degrees)': 'Dec',
        'Dis': 'Dis',
        'e_Dis_min': 'e_dis_min', 
        'e_Dis_max': 'e_dis_max'
    })
    
    # Ajout des colonnes manquantes pour EDD avec valeurs par défaut
    df_edd['ref_V_h'] = 'LEDA'
    df_edd['ref_dis'] = 'CF4'
    df_edd['Source'] = np.nan
    df_edd['Type'] = np.nan
    
    # Réorganisation des colonnes pour correspondre à CenA
    colonnes_finales = ['Name', 'PGC', 'RA', 'Dec', 'Dis', 'e_dis_min', 'e_dis_max', 
                       'ref_dis', 'V_h', 'e_V_h', 'ref_V_h', 'Source', 'Type']
    
    df_edd = df_edd[colonnes_finales]
    df_cena = df_cena[colonnes_finales]
    
    # Identification des PGC communs
    pgc_cena = set(df_cena['PGC'].dropna())
    pgc_edd = set(df_edd['PGC'].dropna())
    pgc_communs = pgc_cena.intersection(pgc_edd)
    
    print(f"PGC communs trouvés: {len(pgc_communs)}")
    
    # Filtrage de CenA: garde seulement les lignes sans PGC ou avec PGC non présent dans EDD
    df_cena_filtre = df_cena[
        (df_cena['PGC'].isna()) | 
        (~df_cena['PGC'].isin(pgc_communs))
    ]
    
    # Affichage des galaxies sans PGC ajoutées
    galaxies_sans_pgc_cena = df_cena_filtre[df_cena_filtre['PGC'].isna()]['Name'].tolist()
    galaxies_sans_pgc_edd = df_edd[df_edd['PGC'].isna()]['Name'].tolist()
    
    if galaxies_sans_pgc_cena:
        print("Galaxies sans PGC ajoutées depuis CenA_dwarfs.csv:")
        for nom in galaxies_sans_pgc_cena:
            print(f"  - {nom}")
    
    if galaxies_sans_pgc_edd:
        print("Galaxies sans PGC ajoutées depuis EDD_data_TRGB.csv:")
        for nom in galaxies_sans_pgc_edd:
            print(f"  - {nom}")
    
    # Fusion des DataFrames
    df_final = pd.concat([df_cena_filtre, df_edd], ignore_index=True)
    
    # Tri par Name pour une meilleure lisibilité
    df_final = df_final.sort_values('Name').reset_index(drop=True)
    
    # Sauvegarde
    df_final.to_csv(fichier_sortie, index=False)
    
    print(f"\nFusion terminée!")
    print(f"Fichier final: {len(df_final)} lignes")
    print(f"Sauvegardé dans: {fichier_sortie}")
    
    return df_final

# Exemple d'utilisation
if __name__ == "__main__":
    # Exemple d'appel de la fonction
    df_resultat = fusionner_csv_galaxies(
        "CenA_dwarfs.csv", 
        "EDD_data_TRGB_modified.csv", 
        "galaxies_fusionnees.csv"
    )

Lecture des fichiers CSV...
CenA_dwarfs.csv: 28 lignes
EDD_data_TRGB.csv: 143 lignes
PGC communs trouvés: 0
Galaxies sans PGC ajoutées depuis CenA_dwarfs.csv:
  - dw1322-39
  - dw1323-40b
  - dw1323-40a
  - dw1341-43
  - dw1342-43

Fusion terminée!
Fichier final: 171 lignes
Sauvegardé dans: galaxies_fusionnees.csv
